In [2]:
import pandas as pd
import numpy as np
from epiweeks import Week
from numpy import log, exp
from scipy.stats import lognorm
from scipy.optimize import minimize
from matplotlib import use, get_backend
import aux_functions

#use('TkAgg', force=True)
from matplotlib import pyplot as plt
#print("Switched to:", get_backend())

Load preds: 

In [4]:
df_preds= pd.read_csv('results/forecast_quantiles_death_v3.csv')

#df_preds = df_preds.loc[df_preds.model_id != 141]
df_preds.forecast_date = pd.to_datetime(df_preds.forecast_date)
df_preds.target_end_date = pd.to_datetime(df_preds.target_end_date)


#df_preds = df_preds.loc[df_preds.date <= Week(2025, 40).startdate().strftime('%Y-%m-%d')]
            
df_preds.head()

,model,forecast_date,target_end_date,horizon,lower_95,upper_95,lower_50,upper_50,lower_98,lower_90,...,upper_10,upper_20,upper_30,upper_40,upper_60,upper_70,upper_80,upper_90,upper_98,value
0,AIpert-pwllnod,2021-01-11,2021-01-23,1,15585.124720,30107.231540,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21731.0
1,AIpert-pwllnod,2021-01-11,2021-01-30,2,15873.523490,36138.769513,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21862.0
2,AIpert-pwllnod,2021-01-11,2021-02-06,3,16301.068910,44059.836613,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21394.0
3,AIpert-pwllnod,2021-01-11,2021-02-13,4,16888.656010,54505.710936,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17536.0
4,AIpert-pwllnod,2021-01-18,2021-01-30,1,17927.563802,33673.501984,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21862.0


In [5]:
len(df_preds)

11165

In [6]:
df_preds = df_preds.dropna()
df_preds = df_preds.reset_index(drop=True)

In [7]:
len(df_preds)

10933

In [8]:
11165-10933

232

### Estimate from the KL: 

In [9]:
l_pos = tuple([col for col in df_preds.columns if 'lower' in col])
u_pos = tuple([col for col in df_preds.columns if 'upper' in col])
the_pos = list(l_pos + u_pos)
l_pos = list(l_pos)
u_pos = list(u_pos)

lvls = (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.98)
p = [aux_functions.quantile_pair(lvl) for lvl in lvls]
p = [i for quantile in p for i in quantile]
p.append(0.5)
ps = p.copy()
ps.sort()


In [10]:
ps

[0.010000000000000009,
 0.025000000000000022,
 0.04999999999999999,
 0.09999999999999998,
 0.15000000000000002,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9,
 0.95,
 0.975,
 0.99]

In [11]:
the_pos

['lower_95',
 'lower_50',
 'lower_98',
 'lower_90',
 'lower_80',
 'lower_70',
 'lower_60',
 'lower_40',
 'lower_30',
 'lower_20',
 'lower_10',
 'upper_95',
 'upper_50',
 'upper_10',
 'upper_20',
 'upper_30',
 'upper_40',
 'upper_60',
 'upper_70',
 'upper_80',
 'upper_90',
 'upper_98']

In [12]:
l_pos

['lower_95',
 'lower_50',
 'lower_98',
 'lower_90',
 'lower_80',
 'lower_70',
 'lower_60',
 'lower_40',
 'lower_30',
 'lower_20',
 'lower_10']

In [13]:
k = 0
lowers = list(df_preds.loc[k, l_pos])
J = len(lowers)
names_l = [name.split('_') for name in df_preds.loc[k, l_pos].index]
extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]

uppers = list(df_preds.loc[k, u_pos])

intervals = pd.DataFrame({
    'level': [i / 100 for i in extracted_lvls],
    'lwr': lowers,
    'med': [df_preds.loc[k, 'pred']] * J,
    'upr': uppers
})

approxs = [[], []]
for j in range(0, J):
    aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           conf_level = intervals['level'][j],
                                           fn_loss = 'lower')
    approxs[0].append(aux[0])
    approxs[1].append(aux[1])

ln_approx = pd.DataFrame({
    'level': list(intervals['level']),
    'mu': approxs[0],
    'sigma': approxs[1]
})

ln_approx.head()

,level,mu,sigma
0,0.95,7.383117,0.506617
1,0.50,7.634469,0.454866
2,0.98,6.867973,0.347599
3,0.90,7.228791,0.313667
4,0.80,7.399763,0.320556


aux = aux_functions.get_lognormal_pars(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           alpha = intervals['level'][j])

aux

aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                           lwr = intervals['lwr'][j],
                                           upr = intervals['upr'][j],
                                           conf_level = intervals['level'][j], fn_loss = 'median')

aux

print(intervals['lwr'][j], intervals['med'][j], intervals['upr'][j])

print(lognorm.ppf([0.25, 0.5, 0.75], scale = exp(7.9415914959428235),  s = 3.6224374131147576))
print(lognorm.ppf([0.25, 0.5, 0.75], scale = exp(6.74957983380822),  s = 5.389716825671165))


In [14]:
Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
                                               x0 = np.array([0.0, 0.0]),
                                               J = J,
                                               ln_approx = ln_approx)

direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]


direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]


In [15]:
direct_KL_pars

[np.float64(7.370375358086094), np.float64(0.4526838593649469)]

In [16]:
direct_KL_pars_closedForm

[7.370375365421382, np.float64(0.452683862555125)]

In [17]:
ln_approx

,level,mu,sigma
0,0.95,7.383117,0.506617
1,0.50,7.634469,0.454866
2,0.98,6.867973,0.347599
3,0.90,7.228791,0.313667
4,0.80,7.399763,0.320556
5,0.70,7.507668,0.354968
6,0.60,7.627476,0.454140
7,0.40,7.740119,0.655657
8,0.30,7.813051,0.926799
9,0.20,7.897545,1.531982


In [18]:
df_preds.loc[0]

model                   AIpert-pwllnod
forecast_date      2021-07-19 00:00:00
target_end_date    2021-07-31 00:00:00
horizon                              1
lower_95                    595.955733
upper_95                   4341.901891
lower_50                   1521.826927
upper_50                   2810.933415
lower_98                    428.091306
lower_90                    822.916022
lower_80                   1084.586699
lower_70                   1261.134662
lower_60                   1401.449387
lower_40                   1629.929678
lower_30                   1730.102988
lower_20                   1825.157663
lower_10                   1917.124161
pred                       2007.632565
upper_10                   2157.292166
upper_20                   2309.362789
upper_30                   2466.539847
upper_40                   2632.180792
upper_60                   3009.982937
upper_70                   3241.999466
upper_80                   3533.929238
upper_90                 

In [19]:
lognorm.ppf(ps, scale = exp(direct_KL_pars_closedForm[0]),  s = direct_KL_pars_closedForm[1])

array([ 554.06124111,  654.01566151,  754.28982107,  889.12483961,
        993.4648639 , 1085.05614871, 1170.3343311 , 1252.61383593,
       1334.01304911, 1416.13869826, 1500.40522642, 1588.22983773,
       1681.1951685 , 1781.23373124, 1890.89156148, 2013.76828604,
       2155.34480227, 2324.74054034, 2539.06716697, 2837.03019539,
       3344.17083061, 3856.90154826, 4552.69892623])

Condições fixas para todo o método: 

out['est'] = lognorm.ppf(ps,
                              scale = exp(cdf_fit[0]),  # scale = exp(meanlog)
                              s = cdf_fit[1])

out.head(10)

In [20]:
def process_row(row):
    # pegar valores da linha
    lowers = list(row[l_pos])
    J = len(lowers)
    names_l = [name.split('_') for name in row[l_pos].index]
    extracted_lvls = [float(parts[1]) for parts in names_l if len(parts) > 1]
    
    uppers = list(row[u_pos])
    
    intervals = pd.DataFrame({
        'level': [i / 100 for i in extracted_lvls],
        'lwr': lowers,
        'med': [row['pred']] * J,
        'upr': uppers
    })
    
    approxs = [[], []]
    for j in range(0, J):
        aux = aux_functions.get_lognormal_pars2(med = intervals['med'][j],
                                               lwr = intervals['lwr'][j],
                                               upr = intervals['upr'][j],
                                               conf_level = intervals['level'][j],
                                               fn_loss = 'lower')
        approxs[0].append(aux[0])
        approxs[1].append(aux[1])
    
    ln_approx = pd.DataFrame({
        'level': list(intervals['level']),
        'mu': approxs[0],
        'sigma': approxs[1]
    })

    #Opt_direct_KL = aux_functions.minimize_opt_fn1(method = 'L-BFGS-B',
    #                                           x0 = np.array([0.0, 0.0]),
    #                                           J = J,
    #                                           ln_approx = ln_approx)

    #direct_KL_pars = [Opt_direct_KL.x[0], exp(Opt_direct_KL.x[1])]

    direct_KL_pars_closedForm = [sum(ln_approx['mu']/ln_approx['sigma']**2)/sum(1/ln_approx['sigma']**2), np.sqrt(J/sum(1/ln_approx['sigma']**2))]
    
    return pd.Series({
    'mu': direct_KL_pars_closedForm[0],
    'sigma': direct_KL_pars_closedForm[1]
})

In [21]:
%%time 

df_preds[['mu', 'sigma']] = df_preds.apply(process_row, axis=1)

df_preds.head()

C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
C:\Users\alp19\anaconda3\envs\D_M2\Lib\site-packages\scipy\optimize\_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  attained_loss = abs(upr - tent_qs[1]) / upr
C:\Users\alp19\anaconda3\envs\D_M2\Lib\site-packages\scipy\optimize\_optimize.py:869: RuntimeWarning: invalid value encountered in subtract
  np.max(np.abs(fsim[0] - fsim[1:])) <= fatol):
C:\Users\alp19\Documentos\EMAp-Doutorado\Dropbox\Mosqlimate_COVID_HUB\harmonization_strategies_py_COVID\aux_functions.py:74: RuntimeWarning: divide by zero encoun

CPU times: total: 1h 40min 28s
Wall time: 1h 47min 34s


,model,forecast_date,target_end_date,horizon,lower_95,upper_95,lower_50,upper_50,lower_98,lower_90,...,upper_30,upper_40,upper_60,upper_70,upper_80,upper_90,upper_98,value,mu,sigma
0,AIpert-pwllnod,2021-07-19,2021-07-31,1,595.955733,4341.901891,1521.826927,2810.933415,428.091306,822.916022,...,2466.539847,2632.180792,3009.982937,3241.999466,3533.929238,3966.613137,4778.256172,2571.0,7.370375,0.452684
1,AIpert-pwllnod,2021-07-19,2021-08-07,2,482.710172,5682.092035,1566.049852,3355.332615,342.360199,748.270963,...,2831.924726,3083.665134,3657.847238,4010.464978,4454.138612,5111.729773,6345.261427,3668.0,7.406262,0.550544
2,AIpert-pwllnod,2021-07-19,2021-08-14,3,404.932573,7807.806832,1660.787331,4208.366189,281.569183,712.782261,...,3398.666823,3788.103155,4676.349006,5221.840136,5908.192535,6925.470183,8833.713869,4661.0,7.477282,0.652295
3,AIpert-pwllnod,2021-07-19,2021-08-21,4,352.113552,11195.813238,1810.056893,5541.739899,230.007093,709.501538,...,4269.847943,4881.582321,6276.856667,7133.725041,8211.861115,9809.821186,12807.328527,7304.0,7.583561,0.760635
4,AIpert-pwllnod,2021-07-26,2021-08-07,1,649.222301,3820.828929,1459.009347,2550.466220,466.013035,847.726697,...,2264.696295,2402.141372,2715.632972,2908.154993,3150.391657,3509.422884,4182.905755,3668.0,7.327829,0.409190


In [22]:
df_preds.isnull().sum()

model              0
forecast_date      0
target_end_date    0
horizon            0
lower_95           0
upper_95           0
lower_50           0
upper_50           0
lower_98           0
lower_90           0
lower_80           0
lower_70           0
lower_60           0
lower_40           0
lower_30           0
lower_20           0
lower_10           0
pred               0
upper_10           0
upper_20           0
upper_30           0
upper_40           0
upper_60           0
upper_70           0
upper_80           0
upper_90           0
upper_98           0
value              0
mu                 0
sigma              0
dtype: int64

In [23]:
df_preds[df_preds.isnull().any(axis=1)]

,model,forecast_date,target_end_date,horizon,lower_95,upper_95,lower_50,upper_50,lower_98,lower_90,...,upper_30,upper_40,upper_60,upper_70,upper_80,upper_90,upper_98,value,mu,sigma


In [24]:
col = 'sigma'
pos_inf_count = np.isposinf(df_preds[col]).sum()

# count only negative inf
neg_inf_count = np.isneginf(df_preds[col]).sum()

# count both +inf and -inf
total_inf_count = np.isinf(df_preds[col]).sum()

print("positive inf:", pos_inf_count)
print("negative inf:", neg_inf_count)
print("total inf:", total_inf_count)

positive inf: 0
negative inf: 0
total inf: 0


In [26]:
df_preds.to_csv('results/preds_KL_death_v3.csv', index = False)